[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/LDA_content.ipynb)

# Latent Dirichlet Allocation (LDA)

In [5]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# ①データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

--2026-02-03 20:37:33--  https://files.grouplens.org/datasets/movielens/ml-10m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65566137 (63M) [application/zip]
Saving to: ‘../data/ml-10m.zip’

ml-10m.zip          100%[===================>]  62.53M  72.5MB/s    in 0.9s    

2026-02-03 20:37:34 (72.5 MB/s) - ‘../data/ml-10m.zip’ saved [65566137/65566137]

Archive:  ../data/ml-10m.zip
   creating: ../data/ml-10M100K/
  inflating: ../data/ml-10M100K/allbut.pl  
  inflating: ../data/ml-10M100K/movies.dat  
  inflating: ../data/ml-10M100K/ratings.dat  
  inflating: ../data/ml-10M100K/README.html  
  inflating: ../data/ml-10M100K/split_ratings.sh  
  inflating: ../data/ml-10M100K/tags.dat  


In [6]:
# ②Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [7]:
# 因子数(トピック数)
factors = 50
# エポック数
n_epochs = 30
#③LDAの学習に必要なデータを切り出す

In [8]:
movie_content = movies.copy()
# tagが付与されていない映画もあるが、genreはすべての映画に付与されている
# tagとgenreを結合したものを映画のコンテンツ情報として似ている映画を探して推薦していく
# tagがない映画に関しては、NaNになっているので、空のリストに変換してから処理をする
movie_content["tag_genre"] = movie_content["tag"].fillna("").apply(list) + movie_content["genre"].apply(list)
movie_content["tag_genre"] = movie_content["tag_genre"].apply(lambda x: list(map(str, x)))

# ④タグとジャンルデータを使って、ldaを学習する
tag_genre_data = movie_content.tag_genre.tolist()

In [9]:
#⑤ ColabのPythonにgensimをインストールする
import sys
!{sys.executable} -m pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 52.8 MB/s eta 0:00:00


In [10]:
from gensim.corpora.dictionary import Dictionary

#⑥LDAのインプットに使うデータの作成
common_dictionary = Dictionary(tag_genre_data)
common_corpus = [common_dictionary.doc2bow(text) for text in tag_genre_data]

In [11]:
import gensim

# ⑦LDAの学習（潜在トピックを洗い出す）
lda_model = gensim.models.LdaModel(
    common_corpus, id2word=common_dictionary, num_topics=factors, passes=n_epochs
)

In [12]:
# topic0の単語一覧（LDAが見つけた潜在トピックを構成する単語を確認）
for token_id, score in lda_model.get_topic_terms(0, topn=10):
    word = common_dictionary.id2token[token_id]
    print(f'word={word}, score={score}')

word=Comedy, score=0.6885727047920227
word=Drama, score=0.15062315762043
word=woody allen, score=0.018927279859781265
word=erlend's dvds, score=0.006777285132557154
word=sure thing, score=0.005992375314235687
word=adoption, score=0.005939619615674019
word=adolescence, score=0.004588007926940918
word=indie, score=0.0038845238741487265
word=writer, score=0.003717107931151986
word=hw comedy, score=0.0034527417737990618


In [13]:
# サムライという単語のトピック(各トピックの所属確率)
lda_model[common_dictionary.doc2bow(['samurai'])]

[(0, np.float32(0.010000054)),
 (1, np.float32(0.010000054)),
 (2, np.float32(0.010000054)),
 (3, np.float32(0.010000054)),
 (4, np.float32(0.010000054)),
 (5, np.float32(0.010000054)),
 (6, np.float32(0.010000054)),
 (7, np.float32(0.010000054)),
 (8, np.float32(0.010000054)),
 (9, np.float32(0.010000054)),
 (10, np.float32(0.010000054)),
 (11, np.float32(0.010000054)),
 (12, np.float32(0.010000054)),
 (13, np.float32(0.010000054)),
 (14, np.float32(0.010000054)),
 (15, np.float32(0.010000054)),
 (16, np.float32(0.010000054)),
 (17, np.float32(0.010000054)),
 (18, np.float32(0.010000054)),
 (19, np.float32(0.5099976)),
 (20, np.float32(0.010000054)),
 (21, np.float32(0.010000054)),
 (22, np.float32(0.010000054)),
 (23, np.float32(0.010000054)),
 (24, np.float32(0.010000054)),
 (25, np.float32(0.010000054)),
 (26, np.float32(0.010000054)),
 (27, np.float32(0.010000054)),
 (28, np.float32(0.010000054)),
 (29, np.float32(0.010000054)),
 (30, np.float32(0.010000054)),
 (31, np.float32(0.0

In [14]:
# ⑧各映画のトピックを格納
lda_topics = lda_model[common_corpus]

# ⑨各映画に最も確率の高いトピックを１つ取り出し格納していく
movie_topics = []
movie_topic_scores = []
for movie_index, lda_topic in enumerate(lda_topics):
    sorted_topic = sorted(lda_topics[movie_index], key=lambda x: -x[1])
    # 最も確率の高いトピック
    movie_topic, topic_score = sorted_topic[0]
    movie_topics.append(movie_topic)
    movie_topic_scores.append(topic_score)
movie_content["topic"] = movie_topics
movie_content["topic_score"] = movie_topic_scores
movie_content

,movie_id,title,genre,tag,tag_genre,topic,topic_score
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[pixar, pixar, pixar, animation, pixar, animat...","[pixar, pixar, pixar, animation, pixar, animat...",10,0.416784
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[for children, game, animals, joe johnston, ro...","[for children, game, animals, joe johnston, ro...",40,0.274245
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[funniest movies, comedinha de velhinhos engra...","[funniest movies, comedinha de velhinhos engra...",6,0.601238
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",[girl movie],"[girl movie, Comedy, Drama, Romance]",8,0.755000
4,5,Father of the Bride Part II (1995),[Comedy],"[steve martin, pregnancy, remake, steve martin...","[steve martin, pregnancy, remake, steve martin...",13,0.422224
...,...,...,...,...,...,...,...
10676,65088,Bedtime Stories (2008),"[Adventure, Children, Comedy]",NaN,"[Adventure, Children, Comedy]",33,0.563280
10677,65091,Manhattan Melodrama (1934),"[Crime, Drama, Romance]",NaN,"[Crime, Drama, Romance]",22,0.454459
10678,65126,Choke (2008),"[Comedy, Drama]","[chuck palahniuk, based on book]","[chuck palahniuk, based on book, Comedy, Drama]",0,0.505002
10679,65130,Revolutionary Road (2008),"[Drama, Romance]",[toplist08],"[toplist08, Drama, Romance]",8,0.673333


In [15]:
from collections import defaultdict
from collections import Counter

# ⑩各ユーザーのレコメンドリストを作成していく
# ユーザーが高く評価した映画が、どのトピックに所属していることが多いかをカウントする
# 一番多いトピックをユーザーの好きなトピックとみなして、そのトピックの映画をおすすめする

movielens_train_high_rating = movielens_train[movielens_train.rating >= 4]
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()

movie_id2index = dict(zip(movie_content.movie_id.tolist(), range(len(movie_content))))
pred_user2items = defaultdict(list)
for user_id, data in movielens_train_high_rating.groupby("user_id"):
    # ユーザーが高く評価した映画
    evaluated_movie_ids = user_evaluated_movies[user_id]
    # 直近閲覧した映画を取得
    movie_ids = data.sort_values("timestamp")["movie_id"].tolist()[-10:]

    movie_indexes = [movie_id2index[id] for id in movie_ids]

    # 直近閲覧した映画のトピックを取得して、出現回数をカウントする
    topic_counter = Counter([movie_topics[i] for i in movie_indexes])
    # 一番出現回数が多かったトピックを取得
    frequent_topic = topic_counter.most_common(1)[0][0]
    # そのトピックの映画の中でもスコアが高いものをおすすめする
    topic_movies = (
        movie_content[movie_content.topic == frequent_topic]
        .sort_values("topic_score", ascending=False)
        .movie_id.tolist()
    )

    for movie_id in topic_movies:
        if movie_id not in evaluated_movie_ids:
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break
pred_user2items

defaultdict(list,
            {1: [5109, 3964, 3611, 2015, 32153, 2043, 48159, 1848, 4563, 2057],
             2: [1196, 5378, 33493, 2628, 6296, 2115, 1653, 316, 172, 1690],
             3: [1150, 5873, 483, 3050, 846, 1090, 8589, 57, 4009, 2375],
             4: [1210, 1196, 260, 5378, 33493, 2628, 6296, 2115, 1653, 172],
             5: [1284, 3629, 1281, 1203, 25766, 3307, 3462, 7579, 28, 5008],
             6: [832, 33819, 8915, 1358, 55946, 6555, 6802, 422, 2608, 57401],
             7: [194, 3629, 1281, 1203, 1103, 25766, 3307, 3462, 7579, 28],
             8: [2924, 2582, 112, 4444, 3972, 2273, 51277, 59103, 60295, 4468],
             9: [1210, 1196, 260, 5378, 33493, 2628, 6296, 2115, 316, 172],
             10: [1933, 1940, 1949, 1943, 8153, 8494, 1935, 1937, 1927, 3467],
             11: [540, 290, 1307, 4992, 5942, 648, 74, 6155, 36525, 5299],
             12: [2951, 26294, 1209, 3681, 502, 1573, 1095, 2401, 4699, 3005],
             13: [1150, 5873, 483, 3050, 846, 1090, 8

### User 2 が高く評価した映画と、それらに割り当てられたトピック

In [16]:
user2_high_rated_movies = movielens_train_high_rating[movielens_train_high_rating.user_id==2]
display(user2_high_rated_movies.merge(movie_content[['movie_id', 'topic', 'topic_score']], on='movie_id', how='left'))

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank,topic,topic_score
0,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0,44,0.483859
1,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0,25,0.800965
2,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0,45,0.355433
3,2,1210,4.0,868245644,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]","[desert, fantasy, sci-fi, space, lucas, gfei o...",10.0,25,0.948165


上記の結果から、user_id=2は`topic`11の映画を多く評価していることがわかります。これに基づいて、システムは`topic`11に属する未評価の映画を推薦します。

### User 2 に推薦された映画と、それらに割り当てられたトピック

In [17]:
user2_recommended_movie_ids = pred_user2items[2]
user2_recommended_movies = movies[movies.movie_id.isin(user2_recommended_movie_ids)]
display(user2_recommended_movies.merge(movie_content[['movie_id', 'topic', 'topic_score']], on='movie_id', how='left').sort_values('topic_score', ascending=False))

,movie_id,title,genre,tag,topic,topic_score
2,1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Sci-Fi]","[lucas, george lucas, george lucas, gfei own i...",25,0.863590
7,5378,Star Wars: Episode II - Attack of the Clones (...,"[Action, Adventure, Sci-Fi]","[lucas, dvd, seen at the cinema, nerdy, sci-fi...",25,0.800751
9,33493,Star Wars: Episode III - Revenge of the Sith (...,"[Action, Adventure, Fantasy, Sci-Fi]","[lucas, divx, space, space opera, space, on co...",25,0.756593
6,2628,Star Wars: Episode I - The Phantom Menace (1999),"[Action, Adventure, Sci-Fi]","[far future, lucas, fantasy, george lucas, luc...",25,0.748509
8,6296,"Mighty Wind, A (2003)","[Comedy, Drama, Musical]","[folk music, christopher guest, mockumentary, ...",25,0.645609
5,2115,Indiana Jones and the Temple of Doom (1984),"[Action, Adventure]","[lucas, want it, dvd collection, harrison ford...",25,0.547427
3,1653,Gattaca (1997),"[Drama, Sci-Fi, Thriller]","[biology, directorial debut, dystopia, eugenic...",25,0.535602
1,316,Stargate (1994),"[Action, Adventure, Sci-Fi]","[egypt, space, time travel, time travel, alien...",25,0.522801
0,172,Johnny Mnemonic (1995),"[Action, Sci-Fi, Thriller]","[cyberpunk, hackers, keanu reeves, memory, sci...",25,0.514977
4,1690,Alien: Resurrection (1997),"[Action, Horror, Sci-Fi]","[not bad sequel, jeunet, action, horror, sci-f...",25,0.509986


このように、推薦された映画の多くも`topic`11に属していることが確認できます。これが、コンテンツベースレコメンデーションの動作原理です。ユーザーが高く評価したコンテンツの主題（トピック）を特定し、その主題に合致する新しいコンテンツを推薦しています。

In [18]:
# user_id=2のユーザーが学習データで、4以上の評価を付けた映画一覧
movielens_train_high_rating[movielens_train_high_rating.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
22,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
24,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
27,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
38,2,1210,4.0,868245644,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]","[desert, fantasy, sci-fi, space, lucas, gfei o...",10.0


In [19]:
# user_id=2に対するおすすめ(2115, 76, 2006)
movies[movies.movie_id.isin([2115, 76, 2006])]

,movie_id,title,genre,tag
75,76,Screamers (1995),"[Action, Sci-Fi, Thriller]","[philip k. dick, artificial intelligence, post..."
1922,2006,"Mask of Zorro, The (1998)","[Action, Adventure, Romance]","[california, mexico, funny, banderas, anthony ..."
2031,2115,Indiana Jones and the Temple of Doom (1984),"[Action, Adventure]","[lucas, want it, dvd collection, harrison ford..."


In [20]:
# 今回は、各ユーザーの評価履歴から１つのトピックをピックアップしましたが、トピックの確率値を利用して、重み付けしながら、アイテムを抽出することも可能です

In [25]:
import pandas as pd

user_ratings_df = pd.read_csv('/content/myselfdata.csv')
display(user_ratings_df.merge(movie_content[['movie_id', 'topic', 'topic_score']], on='movie_id', how='left'))

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank,topic,topic_score
0,-1,4896,5.0,1.770030e+09,Harry Potter and the Sorcerer's Stone (a.k.a. ...,"['Adventure', 'Children', 'Fantasy']","['based on a book', 'franchise', 'adventure', ...",1,25,0.281956
1,-1,5816,5.0,1.770030e+09,Harry Potter and the Chamber of Secrets (2002),"['Adventure', 'Children', 'Fantasy']","['based on a book', 'franchise', 'harry potter...",2,40,0.353910
2,-1,8368,4.0,1.770030e+09,Harry Potter and the Prisoner of Azkaban (2004),"['Adventure', 'Children', 'Fantasy']","['based on a book', 'franchise', 'magic', 'har...",3,40,0.373654
3,-1,40815,4.5,1.770030e+09,Harry Potter and the Goblet of Fire (2005),"['Adventure', 'Fantasy', 'Thriller']","['based on a book', 'big budget', 'franchise',...",4,40,0.366939
4,-1,54001,0.5,1.770030e+09,Harry Potter and the Order of the Phoenix (2007),"['Adventure', 'Drama', 'Fantasy', 'IMAX']","['franchise', 'too short', 'alan rickman', 'br...",5,13,0.217542
5,-1,5971,3.0,1.770030e+09,My Neighbor Totoro (Tonari no Totoro) (1988),"['Animation', 'Children', 'Fantasy']","['anime', 'miyazaki', 'anime', 'hayao miyazaki...",6,49,0.413241
6,-1,5618,5.0,1.770030e+09,Spirited Away (Sen to Chihiro no kamikakushi) ...,"['Adventure', 'Animation', 'Children', 'Fantasy']","['anime', ""holy christ it's fantastic in every...",7,49,0.322939
7,-1,1022,4.5,1.770030e+09,Cinderella (1950),"['Animation', 'Children', 'Fantasy', 'Musical'...","['disney', 'disney', 'library vhs', 'fairy tal...",8,33,0.498545
8,-1,2125,3.0,1.770030e+09,Ever After: A Cinderella Story (1998),"['Comedy', 'Drama', 'Fantasy', 'Romance']","['definitely for girls', 'lovely', 'xvgb', 'fu...",9,27,0.214045
9,-1,63239,5.0,1.770030e+09,Cinderella (1997),"['Children', 'Fantasy', 'Musical', 'Romance']",NaN,10,33,0.404000


In [26]:
user_ratings_df_high_rated = user_ratings_df[user_ratings_df['rating'] >= 4.0]

movie_id2index = dict(zip(movie_content.movie_id.tolist(), range(len(movie_content))))
movie_indexes = [movie_id2index[id] for id in user_ratings_df_high_rated['movie_id']]
# 直近閲覧した映画のトピックを取得して、出現回数をカウントする
topic_counter = Counter([movie_topics[i] for i in movie_indexes])
# 一番出現回数が多かったトピックを取得
frequent_topic = topic_counter.most_common(1)[0][0]
# そのトピックの映画の中でもスコアが高いものをおすすめする
topic_movies = (
    movie_content[movie_content.topic == frequent_topic]
    .sort_values("topic_score", ascending=False)
    .movie_id.tolist()
)

pred_user2items = defaultdict(list)
for movie_id in topic_movies:
    if movie_id not in evaluated_movie_ids:
        pred_user2items[-1].append(movie_id)
pred_user2items

defaultdict(list,
            {-1: [364,
              5109,
              3964,
              3611,
              2015,
              32153,
              2043,
              48159,
              1848,
              4563,
              2057,
              1011,
              45074,
              2141,
              1881,
              5538,
              62376,
              56,
              4154,
              2566,
              126,
              484,
              5218,
              2080,
              47264,
              6889,
              2050,
              1282,
              3759,
              2687,
              1010,
              2092,
              4366,
              588,
              1064,
              5539,
              47721,
              6371,
              5448,
              5683,
              8224,
              313,
              2800,
              244,
              3483,
              1583,
              4075,
              2876,
              5362,


In [27]:
#紹介された映画の詳細データを抽出
recommended_movies = movie_content[
    movie_content['movie_id'].isin(pred_user2items[-1])
]

display(recommended_movies[['movie_id', 'title', 'genre', 'topic', 'topic_score']])

,movie_id,title,genre,topic,topic_score
7,8,Tom and Huck (1995),"[Adventure, Children]",33,0.505000
12,13,Balto (1995),"[Animation, Children]",33,0.404066
47,48,Pocahontas (1995),"[Animation, Children, Musical, Romance]",33,0.637318
53,54,"Big Green, The (1995)","[Children, Comedy]",33,0.275611
55,56,Kids of the Round Table (1995),"[Adventure, Children, Fantasy]",33,0.755000
...,...,...,...,...,...
10604,64116,Igor (2008),"[Animation, Comedy]",33,0.368975
10612,64249,Shrek the Halls (2007),"[Adventure, Animation, Comedy, Fantasy]",33,0.574306
10618,64285,Wallace and Gromit in 'A Matter of Loaf and De...,"[Animation, Comedy]",33,0.438768
10641,64652,Delgo (2008),"[Adventure, Animation, Comedy, Fantasy, Romance]",33,0.562827
